# Train full-precision models with PocketFlow

# Clone repos and change dir

In [ ]:
# clone/update the GraduationProject
# if project does not exist, then clone the remote repo to local
# if do exit now, then pull from the remote, both the main project and submodules
import os
os.chdir("/content")

!if [ ! -d "/content/GraduationProject" ]; then \
    cd /content && \
    echo -e "\n❗️Project does not exist, pulling from remote..." && \
    git clone https://github.com/WenjayDu/GraduationProject.git && \
    cd GraduationProject && \
    git submodule update --init --recursive && \
    echo "❗️Done pulling"; \
else \
    echo -e "\n❗Project exists. Start update..." && \
    cd /content/GraduationProject && \
    echo "🚩Now in `pwd`" && \
    git pull && \
    cd module_minc_keras && \
    echo "🚩Now in `pwd`" && \
    git checkout master && \
    git pull && \
    cd ../module_pocketflow && \
    echo "🚩Now in `pwd`" && \
    git checkout master && \
    git pull && \
    echo "❗️Done updating"; \
fi


# Prepare dataset

In [ ]:
os.chdir("/content/GraduationProject/datasets")
# decompress output.tar.bz2 and make mri dataset
!if [ ! -d "./mri" ]; then \
    tar -jxvf /content/GraduationProject/module_minc_keras/data/output.tar.bz2 -C ./ &> /dev/null && \
    mv output mri; \
fi

In [ ]:
# download sorteo.tar.bz2 and make sorteo dataset
!if [ ! -d "./sorteo" ]; then \
    wget https://amnesia.cbrain.mcgill.ca/deeplearning/sorteo.tar.bz2 --no-check-certificate && \
    mkdir sorteo && \
    tar -jxvf sorteo.tar.bz2 -C sorteo &> /dev/null; \
fi

In [ ]:
# prepare datasets
os.chdir("/content/GraduationProject")
from data_processing.prepare_datasets import prepare_dataset
prepare_dataset(dataset_name="sorteo",ratios=[0.6, 0.2])

# convert generated .npy files to .tfrecords files
from data_processing.convert_npy_to_tfrecords import convert_whole_dataset
convert_whole_dataset(dataset_dir_path="datasets/sorteo_pad_4")


# preparation work for model compressing

In [ ]:
!if [ ! -d "/content/GraduationProject/module_pocketflow/compressing_with_PF" ]; then \
  cp -r /content/GraduationProject/compressing_with_PF \
  /content/GraduationProject/module_pocketflow/; \
fi

In [ ]:
!if [ ! -d "/content/GraduationProject/module_pocketflow/models" ]; then \
  mkdir /content/GraduationProject/module_pocketflow/models \
fi


In [ ]:
import os
os.chdir("/content/GraduationProject")
from config_and_utils import set_up_output_dirs
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor1')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor2')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor4')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor8')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor16')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor32')
set_up_output_dirs(base_dir='PF_models/full-prec/sorteo_original_divisor64')

set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor1')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor2')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor4')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor8')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor16')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor32')
set_up_output_dirs(base_dir='PF_models/chn-pruned-rmt/sorteo_original_divisor64')

set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor1')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor2')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor4')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor8')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor16')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor32')
set_up_output_dirs(base_dir='PF_models/chn-pruned-gpu/sorteo_original_divisor64')

set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor1')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor2')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor4')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor8')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor16')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor32')
set_up_output_dirs(base_dir='PF_models/uniform-tf/sorteo_original_divisor64')


# full precision training 

## on sorteo dataset

### original_divisor1

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=1 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor1/models/model.tflite

### original_divisor2

In [ ]:
import os
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=2 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor2/models/model.tflite

### original_divisor4

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=4 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor4/models/model.tflite

### original_divisor8

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=8 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor8/models/model.tflite

### original_divisor16

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=16 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor16/models/model.tflite

### original_divisor32

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=32 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor32/models/model.tflite

### original_divisor64

In [ ]:
os.chdir("/content/GraduationProject/module_pocketflow")
! ./scripts/run_local.sh compressing_with_PF/unet_run.py \
    --structure=original \
    --divisor=64 \
    --save_path=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models/model.ckpt \
    --save_path_eval=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models_eval/model.ckpt \
    --log_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/logs \
    --data_dir=/content/GraduationProject/datasets/sorteo_pad_4/data \
    --learner=full-prec \
    --input_shape=[112,112,1] \
    --epoch_num=80 \
    --lrn_rate_init=0.2 \
    --batch_size=8

In [ ]:
! python3 tools/conversion/export_pb_tflite_models.py \
    --model_dir=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models \
    --meta_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models/model.ckpt-5085.meta \
    --scope=model

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models/model.pb

In [ ]:
! python3 tools/benchmark/calc_inference_time.py \
    --model_file=/content/GraduationProject/PF_models/full-prec/sorteo_original_divisor64/models/model.tflite
